In [18]:
%matplotlib widget
%config InlineBackend.figure_format = 'svg'

In [19]:
import copy
import datetime
import os
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload

import celeri
celeri = reload(celeri)

In [20]:
RUN_NAME = datetime.datetime.now().strftime("%y%m%d%H%M%S") + os.sep
command_file_name = './data/western_north_america/western_north_america_command.json'
command, segment, block, meshes, station, mogi, sar = celeri.read_data(command_file_name)
station = celeri.process_station(station, command)
segment = celeri.process_segment(segment, command)
sar = celeri.process_sar(sar, command)
segment, station, block, mogi, sar = celeri.assign_block_labels(segment, station, block, mogi, sar)
assembly = celeri.create_assembly_dictionary()
assembly = celeri.merge_geodetic_data(assembly, station, sar) 

In [21]:
# celeri.plot_block_labels(segment, block, station)

In [22]:
def euler_pole_covariance_to_rotation_vector_covariance(omega_x, omega_y, omega_z, euler_pole_covariance_all):
    """
    This function takes the model parameter covariance matrix
    in terms of the Euler pole and rotation rate and linearly
    propagates them to rotation vector space.
    """
    omega_x_sig = np.zeros_like(omega_x)
    omega_y_sig = np.zeros_like(omega_y)
    omega_z_sig = np.zeros_like(omega_z)

    for i in range(len(omega_x)):
        x = omega_x[i]
        y = omega_y[i]
        z = omega_z[i]
        # current_idx = 3 * i
        # euler_pole_covariance = big_Cov_epoles[current_idx:current_idx + 3 , current_idx:current_idx + 3]
        euler_pole_covariance_current = euler_pole_covariance_all[3*i:3*(i+1), 3*i:3*(i+1)]

        """
        There may be cases where x, y and z are all zero.  This leads to /0 errors.  To avoid this  %%
        we check for these cases and Let A = b * I where b is a small constant (10^-4) and I is     %%
        the identity matrix
        """
        if (x == 0) and (y == 0):
            euler_to_cartsian_operator = 1e-4 * np.eye(3)  # Set a default small value for rotation vector uncertainty
        else:
            # Calculate the partial derivatives
            dlat_dx = -z / (x**2 + y**2)**(3/2) / (1 + z**2 / (x**2 + y**2)) * x
            dlat_dy = -z / (x**2 + y**2)**(3/2) / (1 + z**2 / (x**2 + y**2)) * y
            dlat_dz = 1 / (x**2 + y**2)**(1/2) / (1 + z**2 / (x**2 + y**2))
            dlon_dx = -y / x**2 / (1 + (y / x)**2)
            dlon_dy = 1 / x / (1 + (y / x)**2)
            dlon_dz = 0
            dmag_dx = x / np.sqrt(x**2 + y**2 + z**2)
            dmag_dy = y / np.sqrt(x**2 + y**2 + z**2)
            dmag_dz = z / np.sqrt(x**2 + y**2 + z**2)
            euler_to_cartsian_operator = np.array([[dlat_dx, dlat_dy, dlat_dz],
                                                   [dlon_dx, dlon_dy, dlon_dz],
                                                   [dmag_dx, dmag_dy, dmag_dz]])

        # Propagate the Euler pole covariance matrix to a rotation rate
        # covariance matrix
        rotation_vector_covariance = np.inv(euler_to_cartsian_operator) * euler_pole_covariance_current * np.inv(euler_to_cartsian_operator).T

        # Organized data for the return
        main_diagonal_values = np.diag(rotation_vector_covariance)
        omega_x_sig[i] = np.sqrt(main_diagonal_values[0])
        omega_y_sig[i] = np.sqrt(main_diagonal_values[1])
        omega_z_sig[i] = np.sqrt(main_diagonal_values[2])
    return omega_x_sig, omega_y_sig, omega_z_sig


def get_block_constraint_partials(block):
    """
    Partials for a priori block motion constraints.
    Essentially a set of eye(3) matrices
    """
    apriori_block_idx = np.where(block.apriori_flag.to_numpy() == 1)[0]
    print(apriori_block_idx)
    operator = np.zeros((3 * len(apriori_block_idx), 3 * len(block)))
    for i in range(len(apriori_block_idx)):
        start_row = 3 * i
        start_column = 3 * apriori_block_idx[i]
        operator[start_row:start_row+3, start_column:start_column+3] = np.eye(3)
    return operator


def block_constraints(assembly, block, command):
    """
    Applying a priori block motion constraints
    """
    block_constraint_partials = get_block_constraint_partials(block)
    assembly["index"]["block_constraints_idx"] = np.where(block.apriori_flag.to_numpy == 1)[0]
    assembly["data"]["n_block_constraints"] = 3 * len(assembly["index"]["block_constraints_idx"])
    print(block_constraint_partials.shape[0])
    assembly["data"]["block_constraints"] = np.zeros(block_constraint_partials.shape[0])
    assembly["sigma"]["block_constraints"] = np.zeros(block_constraint_partials.shape[0])
    if assembly["data"]["n_block_constraints"] > 0:
        (
            assembly["data"]["block_constraints"][0::3],
            assembly["data"]["block_constraints"][1::3],
            assembly["data"]["block_constraints"][2::3],
        ) = celeri.sph2cart(
            np.deg2rad(block.euler_lon[assembly["index"]["block_constraints_idx"]]),
            np.deg2rad(block.euler_lat[assembly["index"]["block_constraints_idx"]]),
            np.deg2rad(block.rotation_rate[assembly["index"]["block_constraints_idx"]]),
        )

        euler_pole_covariance_all = np.diag(np.concatenate((np.deg2rad(block.eulerLatSig[assembly["index"]["block_constraints_idx"]]),
                                                            np.deg2rad(block.eulerLonSig[assembly["index"]["block_constraints_idx"]]),
                                                            np.deg2rad(block.rotationRateSig[assembly["index"]["block_constraints_idx"]]))))
        (
            assembly["sigma"]["block_constraints"][0::3],
            assembly["sigma"]["block_constraints"][1::3],
            assembly["sigma"]["block_constraints"][2::3],
        ) = euler_pole_covariance_to_rotation_vector_covariance(
            assembly["data"]["block_constraints"][0::3],
            assembly["data"]["block_constraints"][1::3],
            assembly["data"]["block_constraints"][2::3],
            np.diag(euler_pole_covariance_all)
        )

    assembly["sigma"]["sigma.block_constraint_weight"] = command["block_constraint_weight"]
    return assembly, block_constraint_partials 

assembly, block_constraint_partials = block_constraints(assembly, block, command)

[29]
3


In [49]:
a = np.array([[1, 2, 3],[4, 5, 6,],[7, 8, 9]])
print(a)
print(np.diag(a))
print(np.diag(np.diag(a)))



[[1 2 3]
 [4 5 6]
 [7 8 9]]
[1 5 9]
[[1 0 0]
 [0 5 0]
 [0 0 9]]


In [46]:
b = zip(*a[:])

In [47]:
print(b)